<h1>CS152 Assignment 1: The 8-puzzle</h1>

Before you turn in this assignment, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then run the test cells for each of the questions you have answered.  Note that a grade of 3 for the A* implementation requires all tests in the "Basic Functionality" section to be passed.  The test cells pass if they execute with no errors (i.e. all the assertions are passed).

Make sure you fill in any place that says `YOUR CODE HERE`.  Be sure to remove the `raise NotImplementedError()` statements as you implement your code - these are simply there as a reminder if you forget to add code where it's needed.

---

<h1>
Question 1    
</h1>
Define your <code>PuzzleNode</code> class below.  Ensure that you include all attributes that you need to implement an A* search.  If you wish, you can even include member functions, such as a function to generate successor states.  Alternatively, you can code up this functionality later in the <code>solvePuzzle</code> function.

In [1]:
# Import any packages you need here
# Also define any variables as needed
import copy

#Now, define the class PuzzleNode:
class PuzzleNode:
    """
    Class PuzzleNode: Provides a structure for performing A* search for the n^2-1 puzzle
    """
    
    # initialization
    def __init__(self, state, gval=0, fval=0, parent=None):                    
        self.state = state
        self.parent = parent
        self.pruned = False # flag for pruning
         
        self.G = gval # dist from start node
        self.F = fval # g+h
        
        
        self.dims = (len(state)-1, len(state[0])-1)
    
    # for comparison in priority queue
    def __lt__(self,other):
        return self.F < other.F
    
    # pretty print state, for debugging and display
    def __str__(self):
        res = "_____\n"
        for row in self.state:
            res += ' '.join(map(str, row))
            res += '\n'
        res += "_____\n"
        return res
    
    def next_states(self):
        # find the empty space
        zr = zc = -1
        for i, row in enumerate(self.state):
            try:
                zr, zc = i, row.index(0)
            except ValueError:
                pass
        
        nextStates = []
        
        # make children with one move diff        
        if zr > 0: # if neighbour exists
            state = copy.deepcopy(self.state) # make a copy, not a reference 
            state[zr][zc], state[zr-1][zc] = state[zr-1][zc], state[zr][zc] #swap w 0
            nextStates.append(state)
        if zr < self.dims[0]:
            state = copy.deepcopy(self.state)
            state[zr][zc], state[zr+1][zc] = state[zr+1][zc], state[zr][zc]
            nextStates.append(state)
        if zc > 0 :
            state = copy.deepcopy(self.state)
            state[zr][zc], state[zr][zc-1] = state[zr][zc-1], state[zr][zc]
            nextStates.append(state)
        if zc < self.dims[1]:
            state = copy.deepcopy(self.state)
            state[zr][zc], state[zr][zc+1] = state[zr][zc+1], state[zr][zc] 
            nextStates.append(state)
        return(nextStates)
    


<h1>
Question 2    
</h1>
Define your heuristic functions using the templates below.  Ensure that you extend the <code>heuristics</code> list to include all the heuristic functions you implement.  Note that state will be given as a list of lists, so ensure your function accepts this format.  You may use packages like numpy if you wish within the functions themselves.

In [2]:
# Add any additional code here (e.g. for the memoization extension)

# helper for mapping a "count" function to every tile
def traverse_puzzle(state, f):
    count = 0
    for r, row in enumerate(state):
        for c, t in enumerate(row):
            count += f(t,r,c)
    return(count)

# Misplaced tiles heuristic
def h1(state):
    """
    This function returns the number of misplaced tiles, given the board state
    Input:
        -state: the board state as a list of lists
    Output:
        -h: the number of misplaced tiles
    """    
    # if tile is not 0 and val != flattened index, +1
    # shortcut: True type casts to 1
    return(traverse_puzzle(state, lambda t,r,c: t != 0 and t!= r*3+c))

# Manhattan distance heuristic
def h2(state):
    """
    This function returns the Manhattan distance from the solved state, given the board state
    Input:
        -state: the board state as a list of lists
    Output:
        -h: the Manhattan distance from the solved configuration
    """

    # if tile is not zero, add row diff and col diff
    return(traverse_puzzle(state, lambda t,r,c: (t != 0) * (abs(t//3-r) + abs(t%3-c))))
    
# EXTENSION: <manhattan distance> + 2*<linear conflict>
'''
linear conflict is defined as when tile A is in the path of the goal position of tile A' and vice versa.
This is admissable because for any linear conflict, we will need to make at least
two additional moves: to get one of the tiles out of the way and then back into place.

Two tiles tj and tk are in linear conflict if
    - they are on the same col
    - their goal positions are both in that col
    - tj is to the right of tk # to prevent double counting
    - the goal position of tj is to the left of tk

Reference: https://www.cs.princeton.edu/courses/archive/fall14/cos226/checklist/8puzzle.html
    
'''

def h3(state):
    n = len(state)
    
    # ____ HELPER FUNCTIONS ____
    def goal(v):
        return(v//n, v%n)
    def pos(i):
        return(i//n, i%n)
    
    manh = h2(state)
    
    # store largest tile seen so far in each line 
    # stored value is tk, helps prevent multiple counting on a single tile
    maxr, maxc = {}, {} 

    num_lc = 0
    
    # traversal in increasing index value
    for r, row in enumerate(state):
        for c, v in enumerate(row):
            # v is tj
            if v == 0:
                continue # ignore the 0 tile
            
            gr, gc = v//n, v%n # find the goal position
            
            if r == gr: # if currently in goal row
                # note: comparing col is equivalent to comparing val
                #       since the row is the same
                
                if r in maxr and maxr[r] > gc:
                    # since tj's val is smaller
                    #    its goal position is on the left of tk
                    # but since tk was seen before tj
                    #    tj is to the right of tk
                    # :. Linear conflict found
                    num_lc += 1

                elif r not in maxr or maxr[r] < gc:
                    # else, update max for this row 
                    # if current val is larger
                    maxr[r] = gc
                    
            elif c == gc: # repeat for column lines
                if c in maxc and maxc[c] > gr:                
                    num_lc += 1

                elif c not in maxc or gr > maxc[c]:
                    maxc[c] = gr
    
    return(manh + 2*num_lc)
    
# If you implement more than 3 heuristics, then add any extra heuristic functions onto the end of this list.
heuristics = [h1, h2, h3]

<h1>
Question 3    
</h1>
Code up your A* search using the SolvePuzzle function within the template below.  Please do not modify the function header, otherwise the automated testing will fail.  You may define other functions or import packages as needed in this cell or by adding additional cells.

In [3]:
# creating child nodes for valid next moves

# Import any packages or define any helper functions you need here

from queue import PriorityQueue
from functools import reduce

# Main solvePuzzle function.
def solvePuzzle(state, heuristic):
    """This function should solve the n**2-1 puzzle for any n > 2 (although it may take too long for n > 4)).
    Inputs:
        -state: The initial state of the puzzle as a list of lists
        -heuristic: a handle to a heuristic function.  Will be one of those defined in Question 2.
    Outputs:
        -steps: The number of steps to optimally solve the puzzle (excluding the initial state)
        -exp: The number of nodes expanded to reach the solution
        -max_frontier: The maximum size of the frontier over the whole search
        -opt_path: The optimal path as a list of list of lists.  That is, opt_path[:,:,i] should give a list of lists
                    that represents the state of the board at the ith step of the solution.
        -err: An error code.  If state is not of the appropriate size and dimension, return -1.  For the extention task,
          if the state is not solvable, then return -2
    """
    
    # _____ HELPER FUNCTIONS _____
    # validates the start state, returning the appropriate error code
    def error_code(state):
        # tiny helper for readability
        def even(x):
            return(x%2 == 0)
        
        n = len(state)
        
        # dimension n x n
        if n == 0 or type(state[0]) != list:
            return(-1)
        elif n != len(state[0]):
            return(-1)
        
        # contains every number from 0 to n**2-1
        seen_ind = [False] * n**2 # seen_ind[i] represents if value i has been seen
        
        for row in state:
            for t in row:
                seen_ind[t] = True 
        
        if sum(seen_ind) != len(seen_ind): # if not every index is True
            return(-1)
            
        # EXTENSION: solvability
        '''
        Let inversion be defined as the number of "out of order" pairs
        Ex: An (A,B) pair is inverted if the value of A is more than B,
            but when the state is flattened into a single list
            the index of A is lower than the index of B.
        
        Reasoning for using inversions:
        Let n be the width of the board
        An n-puzzle only has two types of moves: a horizontal or vertical swap
        A horizontal swap does not change the number of inversions in the state
        A vertical swap will change the inversions on the tiles between its start 
        state and end state by exactly 1 (the tile and itself form one pair), 
        thus it will change the number of inversions by exactly (n-1).
        
        For any odd numbered n (such as 3), (n-1) will be even.
        Thus, a start state is solvable iff it has an even number of inversions.

        For an even numbered n (such as 4), (n-1) will be odd.
        Thus, everytime a tile moves, the parity of inversions will flip.
        Everytime a tile moves, the parity of the blank tile's row distance 
            (including the row it is currently on) from the bottom will also flip.
        Thus, the parity of a blank tile and the parity of the number of inversions
            will always be equal.
        Because we ultimately want to end up with 0 inversions and the blank tile n
            away from the bottom row, we need the number of inversions and the blank tile's
            distance to be equal in order to be solvable.
        
        Reference: https://www.cs.bham.ac.uk/~mdr/teaching/modules04/java2/TilesSolvability.html
        '''
        # flatten for easier two pointer traversal
        flat = reduce(lambda z, y :z + y, state) 
        
        # count number of inversions 
        # and zero's row distance from bottom
        inv = 0 
        rz = -1
        
        for i in range(n**2):                 
            for j in range(i + 1, n**2):
                if flat[i] == 0:
                    rz = i//n
                elif flat[j] == 0:
                    rz = j//n
                elif flat[i] > flat[j]: 
                    inv += 1
        
        # check solvability according to logic above
        if not even(n) and not even(inv):
            return(-2)
        elif even(n) and even(rz) and not even(inv):
            return(-2)
        elif even(n) and not even(rz) and even(inv):
            return(-2)

        return(0) # else, all good
    
    # helper function to create goal state, for goal checking
    def create_goal(state):
        v = 0
        goal = []
        for r, row in enumerate(state):
            arow = []
            for c, _ in enumerate(row):
                arow.append(v)
                v += 1
            goal.append(arow)
        return(goal)
    
    # tuple-fy states, since dictionary keys must be immutable
    def immortalize(lst_of_lst):
        return(tuple(tuple(x) for x in lst_of_lst))

    # _____ VALIDATE INPUT _____
    err = error_code(state)
    if err != 0:
        return(0,0,0,None,err)
    
    # _____ INIT _____
    steps = exp = max_frontier = 0
    opt_path = [state]
    goal = create_goal(state)
    
    # EXTENSION: MEMOIZE H
    hlookup = {}
    imStart = immortalize(state)
    hlookup[imStart] = heuristic(state)
    
    # create start node
    start_node = PuzzleNode(state)
    start_node.F = hlookup[imStart] # ignoring G since g(s) = 0
        
    # priority queue, based on F value as defined in __lt__
    frontier = PriorityQueue()
    frontier.put(start_node) 
    
    # lookup for best paths found to a state
    clookup = {}
    clookup[immortalize(state)] = start_node
    
    # expand frontier
    while not frontier.empty():
        max_frontier = max(max_frontier, frontier.qsize())
        cur_node = frontier.get()
        
        if cur_node.pruned:
            continue
            
        if cur_node.state == goal:
            break
        
        for state in cur_node.next_states():
            G = cur_node.G + 1 # assuming all path costs are 1
            
            # EXTENSION: MEMOIZE H
            imState = immortalize(state) # tuple-fy for dictionary keys
            if imState not in hlookup:
                hlookup[imState] = heuristic(state)
            H = hlookup[imState]

            if imState in clookup:
                if clookup[imState].G <= G:
                    # if better path to this state is known, ignore this path
                    continue
                else:
                    # if previously known path is worse, discard it
                    clookup[imState].pruned = True 
            
            # add new node to frontier, update lookup
            child_node = PuzzleNode(state, G, G + H, cur_node)
            frontier.put(child_node)
            clookup[imState] = child_node                   
        exp += 1
    
    # recreate optimal path, count steps
    while cur_node.parent is not None:
        opt_path.insert(1, cur_node.state)
        cur_node = cur_node.parent
        steps += 1
    return(steps, exp, max_frontier, opt_path, err)


<h1>Extension Questions</h1>

The extensions can be implemented by modifying the code from Q2-3 above appropriately.

1. <b>Initial state solvability:</b>  Modify your SolvePuzzle function code in Q3 to return -2 if an initial state is not solvable to the goal state.
2. <b>Extra heuristic function:</b> Add another heuristic function (e.g. pattern database) that dominates the misplaced tiles and Manhattan distance heuristics to your Q2 code.
3. <b>Memoization:</b>  Modify your heuristic function definitions in Q2 by using a Python decorator to speed up heuristic function evaluation

There are test cells provided for extension questions 1 and 2.

<h1>Basic Functionality Tests</h1>
The cells below contain tests to verify that your code is working properly to be classified as basically functional.  Please note that a grade of <b>3</b> on #aicoding and #search as applicable for each test requires the test to be successfully passed.  <b>If you want to demonstrate some other aspect of your code, then feel free to add additional cells with test code and document what they do.<b>

In [4]:
## Test for state not correctly defined

incorrect_state = [[0,1,2],[2,3,4],[5,6,7]]
_,_,_,_,err = solvePuzzle(incorrect_state, lambda state: 0)
assert(err == -1)

In [5]:
## Heuristic function tests for misplaced tiles and manhattan distance

# Define the working initial states
working_initial_states_8_puzzle = ([[2,3,7],[1,8,0],[6,5,4]], [[7,0,8],[4,6,1],[5,3,2]], [[5,7,6],[2,4,3],[8,1,0]])

# Test the values returned by the heuristic functions
h_mt_vals = [7,8,7]
h_man_vals = [15,17,18]

for i in range(0,3):
    h_mt = heuristics[0](working_initial_states_8_puzzle[i])
    h_man = heuristics[1](working_initial_states_8_puzzle[i])
    assert(h_mt == h_mt_vals[i])
    assert(h_man == h_man_vals[i])


In [ ]:
## A* Tests for 3 x 3 boards
## This test runs A* with both heuristics and ensures that the same optimal number of steps are found
## with each heuristic.

# Optimal path to the solution for the first 3 x 3 state
opt_path_soln = [[[2, 3, 7], [1, 8, 0], [6, 5, 4]], [[2, 3, 7], [1, 8, 4], [6, 5, 0]], 
                 [[2, 3, 7], [1, 8, 4], [6, 0, 5]], [[2, 3, 7], [1, 0, 4], [6, 8, 5]], 
                 [[2, 0, 7], [1, 3, 4], [6, 8, 5]], [[0, 2, 7], [1, 3, 4], [6, 8, 5]], 
                 [[1, 2, 7], [0, 3, 4], [6, 8, 5]], [[1, 2, 7], [3, 0, 4], [6, 8, 5]], 
                 [[1, 2, 7], [3, 4, 0], [6, 8, 5]], [[1, 2, 0], [3, 4, 7], [6, 8, 5]], 
                 [[1, 0, 2], [3, 4, 7], [6, 8, 5]], [[1, 4, 2], [3, 0, 7], [6, 8, 5]], 
                 [[1, 4, 2], [3, 7, 0], [6, 8, 5]], [[1, 4, 2], [3, 7, 5], [6, 8, 0]], 
                 [[1, 4, 2], [3, 7, 5], [6, 0, 8]], [[1, 4, 2], [3, 0, 5], [6, 7, 8]], 
                 [[1, 0, 2], [3, 4, 5], [6, 7, 8]], [[0, 1, 2], [3, 4, 5], [6, 7, 8]]]

astar_steps = [17, 25, 28]
for i in range(0,3):
    steps_mt, expansions_mt, _, opt_path_mt, _ = solvePuzzle(working_initial_states_8_puzzle[i], heuristics[0])
    steps_man, expansions_man, _, opt_path_man, _ = solvePuzzle(working_initial_states_8_puzzle[i], heuristics[1])
    # Test whether the number of optimal steps is correct and the same
    assert(steps_mt == steps_man == astar_steps[i])
    # Test whether or not the manhattan distance dominates the misplaced tiles heuristic in every case
    assert(expansions_man < expansions_mt)
    # For the first state, test that the optimal path is the same
    if i == 0:
        assert(opt_path_mt == opt_path_soln)


In [ ]:
## A* Test for 4 x 4 board
## This test runs A* with both heuristics and ensures that the same optimal number of steps are found
## with each heuristic.

working_initial_state_15_puzzle = [[1,2,6,3],[0,9,5,7],[4,13,10,11],[8,12,14,15]]
steps_mt, expansions_mt, _, _, _ = solvePuzzle(working_initial_state_15_puzzle, heuristics[0])
steps_man, expansions_man, _, _, _ = solvePuzzle(working_initial_state_15_puzzle, heuristics[1])
# Test whether the number of optimal steps is correct and the same
assert(steps_mt == steps_man == 9)
# Test whether or not the manhattan distance dominates the misplaced tiles heuristic in every case
assert(expansions_mt >= expansions_man)

<h1>Extension Tests</h1>
The cells below can be used to test the extension questions.  Memoization if implemented will be tested on the final submission - you can test it yourself by testing the execution time of the heuristic functions with and without it.

In [ ]:
## Puzzle solvability test

unsolvable_initial_state = [[7,5,6],[2,4,3],[8,1,0]]
_,_,_,_,err = solvePuzzle(unsolvable_initial_state, lambda state: 0)
assert(err == -2)

In [ ]:
## Extra heuristic function test.  
## This tests that for all initial conditions, the new heuristic dominates over the manhattan distance.

dom = 0
for i in range(0,3):
    steps_new, expansions_new, _, _, _ = solvePuzzle(working_initial_states_8_puzzle[i], heuristics[2])
    steps_man, expansions_man, _, _, _ = solvePuzzle(working_initial_states_8_puzzle[i], heuristics[1])
    # Test whether the number of optimal steps is correct and the same
    assert(steps_new == steps_man == astar_steps[i])
    # Test whether or not the manhattan distance is dominated by the new heuristic in every case, by checking
    # the number of nodes expanded
    dom = expansions_man - expansions_new
assert(dom > 0)

In [ ]:
## Memoization test - will be carried out after submission